<a href="https://colab.research.google.com/github/nakajinbee/zero-tuku-deep-learning-2/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def sigmoid(x):
  return 1 /(1 + np.exp(-x))

x = np.random.randn(10, 2)
w1 = np.random.randn(2, 4)
b1 = np.random.randn(4)
w2 = np.random.randn(4, 3)
b2 = np.random.randn(3)

h = np.dot(x,w1) + b1
print("hの値：")
print(h)
a = sigmoid(h)
print("aの値：")
print(a)
s = np.dot(a, w2) + b2
print("sの値")
print(s)

In [ ]:
# Sigmoidレイア 
class Sigmoid:
  def __init__(self):
    self.params, self.grads = [], []
    self.out = None

  def forward(self, x):
    out = 1/(1 + np.exp(-x))
    self.out = out
    return out
    
  def backward(self, dout):
    dx = dout * (1.0 * self.out) * self.out
    return dx

In [ ]:
# 全結合層レイアであるAffineレイア
class Affine:
  def __init__(self, w, b):
    self.params = [w, b]
    self.grads  = [np.zeros_like(w), np.zeros_like(b)]
  
  def forward(self,x):
    w, b = self.params
    out = np.dot(x, w) + b
    self.x = x
    return out

  def backward(self, dout):
    w, b = self.params
    dx = np.dot(dout, w.T)
    dw = np.dot(self.x.T, dout) 
    db = np.sum(dout, axis=0)

    self.grads[0][...] = dw
    self.grads[1][...] = db
    return dx
    

In [ ]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size):
    I, H, O = input_size, hidden_size, output_size

    # 重みとバイアスの初期化
    self.layers = [
                   Affine(w1, b1),
                   Sigmoid(),
                   Affine(w2, b2)
    ]

    # 全ての重みをリストにまとめる
    self.params = []
    for layer in self.layers:
      self.params += layer.params
    
  def predict(self, x):
    for layer in self.layers:
      x = layer.forward(x)
    return x
  

In [ ]:
x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)

print(s)

In [ ]:
class MatMul:
  def __init__(w):
    self.params = [w]
    self.grads = [np.zeros_like(w)]
    self.x = None

  def forward(self, x):
    w, = self.params
    out = np.dot(x,w)
    self.x = x
    return out

  def backward(self, dout):
    w, = self.params
    dx = np.dot(dout, w.T)
    dw = np.dot(self.x.T, dout)
    self.grads[0][...] = dw
    return dx
    

In [ ]:
class Softmax:

  def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

    return x

    def __init__(self):
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        self.out = softmax(x)
        return self.out

    def backward(self, dout):
        dx = self.out * dout
        sumdx = np.sum(dx, axis=1, keepdims=True)
        dx -= self.out * sumdx
        return dx

誤差逆伝播法によって勾配を求めた
⇨勾配を用いてニューラルネットワークのパラメータを更新する

step1 ミニバッチ：訓練データの中からランダムに複数のデータを選び出す

step2 勾配の算出：誤差逆伝播法により、各重みパラメータに関する損失関数の勾配をもとめる

step3 パラメータの更新：勾配を使って重みパラメータを更新する

step4 繰り返す：１２３を必要な回数だけ繰り返す

In [ ]:
# SDG : Stochastic Gradient Decent (確率的勾配降下法)
# lr : learning rate 学習係数

class SGD:
  def __init__(self, lr=0.01):
    self.lr = lr
  
  def update(self, params, grads):
    for i in range(len(params)):
      params[i] -= self.lr * grads[i]
  